Unzip the Data

In [ ]:
import zipfile
path_to_zip_file = 'data.zip'
directory_to_extract_to = './data/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

Process raw ecg data, clean, R peak detection, outlier r peak removal, feature computation and standardization

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tomkin import detect_rpeak
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from outlier_calculation import Quality,compute_outlier_ecg
# from hrvanalysis import remove_ectopic_beats
from joblib import Parallel,delayed
from data_quality import ECGQualityCalculation
from joblib import delayed,Parallel
from copy import deepcopy
from ecg import ecg_feature_computation
from scipy import interpolate
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler,RobustScaler,QuantileTransformer
import gzip
from scipy import stats

def get_interpolated(aclx,acly,aclz):
    time_array = aclx[:,1].reshape(-1,1)
    aclxyz = np.concatenate([time_array,time_array,time_array,time_array],axis=1)
    f = interpolate.interp1d(aclx[:,1],aclx[:,0],fill_value='extrapolate')
    aclxyz[:,1] = f(aclxyz[:,0])
    f = interpolate.interp1d(acly[:,1],acly[:,0],fill_value='extrapolate')
    aclxyz[:,2] = f(aclxyz[:,0])
    f = interpolate.interp1d(aclz[:,1],aclz[:,0],fill_value='extrapolate')
    aclxyz[:,3] = f(aclxyz[:,0])
    return aclxyz


def get_clean_ecg(ecg_data):
    final_data = np.zeros((0,3))
    if len(ecg_data)==0:
        return final_data
    test_object = ECGQualityCalculation()
    start_ts = ecg_data[0,0]
    final_data = np.zeros((0,3))
    while start_ts<ecg_data[-1,0]:
        index = np.where((ecg_data[:,0]>=start_ts)&(ecg_data[:,0]<start_ts+3000))[0]
        temp_data = ecg_data[index,2]
        if test_object.current_quality(temp_data)==1:
            final_data = np.concatenate((final_data,ecg_data[index,:]))
        start_ts = start_ts + 3000
    return final_data


def get_hr(ecg_data):
#     try:
    rpeaks = detect_rpeak(ecg_data[:,2],64)
    rpeak_ts = ecg_data[rpeaks,0]
    ecg_rr = np.zeros((len(rpeaks)-1,2))
    ecg_rr_ts = np.array(rpeak_ts)[1:]
    ecg_rr_sample = np.array(np.diff(rpeak_ts))
    index = np.where((ecg_rr_sample>=300)&(ecg_rr_sample<=2000))[0]
    ecg_rr_ts = ecg_rr_ts[index]
    ecg_rr_sam = ecg_rr_sample[index]
#     rr = remove_ectopic_beats(ecg_rr_sam)
    rr = ecg_rr_sam
    ecg_rr_sam = ecg_rr_sam[~np.isnan(rr)]
    ecg_rr_ts = ecg_rr_ts[~np.isnan(rr)]
    outlier = compute_outlier_ecg(ecg_rr_ts/1000,ecg_rr_sam/1000)
    ind1 = []
    for ind,tup in enumerate(outlier):
        if tup[1]==Quality.ACCEPTABLE:
            ind1.append(ind)
    ind1 = np.array(ind1)
    if len(ind1)<100:
        return [],[]
    ecg_rr_ts = ecg_rr_ts[ind1]
    ecg_rr_sam = ecg_rr_sam[ind1]
    return ecg_rr_ts,ecg_rr_sam
#     except Exception as e:
#         print(e)


def get_windows(data,window_size=10,offset=10,fs=1):
    ts_array = np.arange(data[0,0],data[-1,0],offset*1000)
    window_col = []
    for t in ts_array:
        index = np.where((data[:,0]>t-window_size*1000/2)&(data[:,0]<=t+window_size*1000/2))[0]
        if len(index)<30:
            continue
        window_col.append(data[index,:])
    return window_col

def get_std_chest(window,start=1,end=4):
    return np.array([np.mean(window[:,0]),np.sqrt(np.sum(np.power(np.std(window[:,start:end],axis=0),2)))])


def filter_ecg_windows(ecg_windows,acl_std):
    final_ecg_windows = []
    for window in ecg_windows:
        index = np.where((acl_std[:,0]>window[0,0])&(acl_std[:,0]<window[-1,0]))[0]
        if len(index)==0:
            continue
        window_temp = acl_std[index,1].reshape(-1)
        if len(window_temp[window_temp>.21])/len(window_temp) > .5:
            continue
        final_ecg_windows.append(window)
    return final_ecg_windows
path = './data/'
participants = [path + f +'/' for f in os.listdir(path) if f[0]=='S']
for f in participants:
    if 'ecg.txt.gz' not in os.listdir(f):
        continue
    st = 0
    et = 0 
    with gzip.open(f+'stress_marks.txt.gz', 'r') as file:
        for line in file.readlines():
            line = line.decode('utf8').strip()
            parts = [x.strip() for x in line.split(',')]
            label = parts[0]
            if label[:2] in ['c1']:
                st = np.int64(parts[2])
                et = np.int64(parts[3])
    aclx = pd.read_csv(f +'accelx.txt.gz', compression='gzip',
                          sep=' ',header=None).values
    acly = pd.read_csv(f +'accely.txt.gz', compression='gzip',
                          sep=' ',header=None).values
    aclz = pd.read_csv(f +'accelz.txt.gz', compression='gzip',
                          sep=' ',header=None).values
    acl_all = get_interpolated(aclx,acly,aclz)
    aclx = ((3*acl_all[:,1].reshape(-1)/4095) - 1.5) / 0.3
    acly = ((3*acl_all[:,2].reshape(-1)/4095) - 1.5) / 0.3
    aclz = ((3*acl_all[:,3].reshape(-1)/4095) - 1.5) / 0.3
    aclxyz = np.concatenate([acl_all[:,0].reshape(-1,1),aclx.reshape(-1,1),acly.reshape(-1,1),aclz.reshape(-1,1)],axis=1)
    acl_windows = get_windows(aclxyz,window_size=10,offset=10,fs=15)
    acl_std = np.array([get_std_chest(window) for window in acl_windows])
    ecg_temp = pd.read_csv(f +'ecg.txt.gz', compression='gzip',
                          sep=' ',header=None).values
    print(ecg_temp.shape,f)
    ecg = np.zeros((ecg_temp.shape[0],ecg_temp.shape[1]+1))
    ecg[:,0],ecg[:,2] = ecg_temp[:,1],ecg_temp[:,0]
    ecg_rr_ts,ecg_rr_sam = get_hr(ecg)
    if len(ecg_rr_sam)<100:
        continue
    print(ecg_rr_ts.shape,f)
    ecg_rr = np.zeros((len(ecg_rr_ts),2))
    ecg_rr[:,0] = ecg_rr_ts
    ecg_rr[:,1] = ecg_rr_sam
    pickle.dump(ecg_rr,open(f+'ecg_rr.p','wb'))
    
    if os.path.isfile(f+'ecg_rr.p') and st>0:
        ecg_rr = pickle.load(open(f+'ecg_rr.p','rb'))
        ecg_rr_baseline = ecg_rr
        ecg_rr[:,1] = stats.mstats.winsorize(ecg_rr[:,1],limits=.1)
        ecg_windows = get_windows(ecg_rr,window_size=60,offset=30,fs=1)
        final_ecg_windows = ecg_windows
        ecg_features = np.array([np.array([window[0,0],window[-1,0]]+ecg_feature_computation(window[:,0],window[:,1])) for window in final_ecg_windows])
        for i in range(2,13,1):
            ecg_features[:,i] = StandardScaler().fit_transform(ecg_features[:,i].reshape(-1,1)).reshape(-1)
        print(len(ecg_windows),len(final_ecg_windows),ecg_features.shape,path,f,ecg_features.shape)
        pickle.dump(ecg_features,open(f+'features22.p','wb'))

(458149, 2) ./data/SI01/
(7412,) ./data/SI01/
238 238 (238, 13) ./data/ ./data/SI01/ (238, 13)
(550199, 2) ./data/SI02/
(9605,) ./data/SI02/
283 283 (283, 13) ./data/ ./data/SI02/ (283, 13)
(476149, 2) ./data/SI03/
(9333,) ./data/SI03/


/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:71: RuntimeWarning: invalid value encountered in double_scalars
  vlf_NU_log = np.log((vlf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:72: RuntimeWarning: invalid value encountered in double_scalars
  lf_NU_log = np.log((lf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:73: RuntimeWarning: invalid value encountered in double_scalars
  hf_NU_log = np.log((hf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:74: RuntimeWarning: invalid value encountered in double_scalars
  lfhfRation_log = np.log((lf_power / hf_power) + 1)


255 255 (255, 13) ./data/ ./data/SI03/ (255, 13)
(460799, 2) ./data/SI04/
(6911,) ./data/SI04/
244 244 (244, 13) ./data/ ./data/SI04/ (244, 13)
(561649, 2) ./data/SI05/
(492299, 2) ./data/SI06/
(10504,) ./data/SI06/
260 260 (260, 13) ./data/ ./data/SI06/ (260, 13)
(505699, 2) ./data/SI07/
(7660,) ./data/SI07/
279 279 (279, 13) ./data/ ./data/SI07/ (279, 13)
(476199, 2) ./data/SI08/
(4208,) ./data/SI08/
126 126 (126, 13) ./data/ ./data/SI08/ (126, 13)
(506049, 2) ./data/SI10/
(8960,) ./data/SI10/


/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:71: RuntimeWarning: invalid value encountered in double_scalars
  vlf_NU_log = np.log((vlf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:72: RuntimeWarning: invalid value encountered in double_scalars
  lf_NU_log = np.log((lf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:73: RuntimeWarning: invalid value encountered in double_scalars
  hf_NU_log = np.log((hf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:74: RuntimeWarning: invalid value encountered in double_scalars
  lfhfRation_log = np.log((lf_power / hf_power) + 1)


271 271 (271, 13) ./data/ ./data/SI10/ (271, 13)
(584349, 2) ./data/SI11/
(496649, 2) ./data/SI12/
(8295,) ./data/SI12/


...

277 277 (277, 13) ./data/ ./data/SI16/ (277, 13)
(502349, 2) ./data/SI17/
(8333,) ./data/SI17/
266 266 (266, 13) ./data/ ./data/SI17/ (266, 13)
(478649, 2) ./data/SI18/
(8379,) ./data/SI18/


/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:71: RuntimeWarning: invalid value encountered in double_scalars
  vlf_NU_log = np.log((vlf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:72: RuntimeWarning: invalid value encountered in double_scalars
  lf_NU_log = np.log((lf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:73: RuntimeWarning: invalid value encountered in double_scalars
  hf_NU_log = np.log((hf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:74: RuntimeWarning: invalid value encountered in double_scalars
  lfhfRation_log = np.log((lf_power / hf_power) + 1)


255 255 (255, 13) ./data/ ./data/SI18/ (255, 13)
(491799, 2) ./data/SI19/
(6035,) ./data/SI19/


/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:71: RuntimeWarning: invalid value encountered in double_scalars
  vlf_NU_log = np.log((vlf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:72: RuntimeWarning: invalid value encountered in double_scalars
  lf_NU_log = np.log((lf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:73: RuntimeWarning: invalid value encountered in double_scalars
  hf_NU_log = np.log((hf_power / (totalPower - vlf_power)) + 1)
/home/jupyter/mullah/Test/data_yield/cstress_features/ecg.py:74: RuntimeWarning: invalid value encountered in double_scalars
  lfhfRation_log = np.log((lf_power / hf_power) + 1)


258 258 (258, 13) ./data/ ./data/SI19/ (258, 13)
(509099, 2) ./data/SI20/
(6659,) ./data/SI20/
263 263 (263, 13) ./data/ ./data/SI20/ (263, 13)
(489149, 2) ./data/SI21/
(8984,) ./data/SI21/
260 260 (260, 13) ./data/ ./data/SI21/ (260, 13)
(472599, 2) ./data/SI22/
(10373,) ./data/SI22/
255 255 (255, 13) ./data/ ./data/SI22/ (255, 13)


In [11]:
# Soujanya Chatterjee
	
# 2:06 PM (9 minutes ago)
	
# to me
import pandas as pd, numpy as np, os, csv, glob, math, matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from datetime import datetime
from scipy.stats import *
import gzip
import pickle
from collections import Counter

def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])

    return maximum[0]

# _dir = 'W:\\Students\\cstress_features\\data\\data\\SI02\\'

def decodeLabel(label):
    label = label[:2]  # Only the first 2 characters designate the label code

    mapping = {'c1': 0, 'c2': 1, 'c3': 1, 'c4': 0, 'c5': 0, 'c6': 0, 'c7': 2}

    return mapping[label]

def readstressmarks(participantID, filename):
    features = []
    for file in os.listdir(filename):    
        if file.endswith("marks.txt.gz"):        
            with gzip.open(os.path.join(filename, file), 'r') as file:
                for line in file.readlines():
                    line = line.decode('utf8').strip()
                    parts = [x.strip() for x in line.split(',')]                    
                    label = parts[0][:2]  
                    if label not in ['c7','c6']:
                        stressClass = decodeLabel(label)
                        features.append([participantID, stressClass, int(parts[2]), int(parts[3])])
    return np.array(features)

_dirr = './data/'
parti = np.array(os.listdir(_dirr) )
print(parti)
header = ['participant','starttime','endtime','label','f_1','f_2','f_3','f_4','f_5','f_6','f_7','f_8','f_9','f_10','f_11']
fea_cols = ['f_1','f_2','f_3','f_4','f_5','f_6','f_7','f_8','f_9','f_10','f_11']
data = []
for p in parti:
    if p in ['feature.csv','feature_ecg.csv','feature_rip.csv','SI05','SI09','SI11','SI14','SI23','SI24','.ipynb_checkpoints']:
        continue
    else:
        if os.path.isdir(os.path.join(_dirr,p)):
           
            _dir = (os.path.join(_dirr,p))
            gt_marks = readstressmarks(p,_dir)
            groundtruth = pd.DataFrame({'participant': gt_marks[:, 0], 'label': gt_marks[:, 1], 'starttime': gt_marks[:, 2],
                                        'endtime': gt_marks[:, 3]}, columns=['participant','label','starttime','endtime'])
            groundtruth = groundtruth.sort_values('starttime')
   

            for file in os.listdir(_dir):    
                    if file.endswith("22.p"):                    
                        with open(_dir+'/'+file, 'rb') as f:  
                            x = pickle.load(f)
#             print(x)
            dataset = pd.DataFrame({'starttime': x[:, 0], 'endtime': x[:, 1], 'f_1': x[:, 2]
                                   , 'f_2': x[:, 3], 'f_3': x[:, 4], 'f_4': x[:, 5]
                                   , 'f_5': x[:, 6], 'f_6': x[:, 7], 'f_7': x[:, 8]
                                   , 'f_8': x[:, 9], 'f_9': x[:, 10], 'f_10': x[:, 11]
                                   , 'f_11': x[:, 12]}, columns=['starttime','endtime','f_1','f_2','f_3','f_4','f_5','f_6','f_7','f_8',
                                                                 'f_9','f_10','f_11'])
           
            dataset = dataset.sort_values('starttime')

            for gt in range(len(dataset)):
                starttime = int(dataset['starttime'].iloc[gt])
                endtime = int(dataset['endtime'].iloc[gt])
                result = []
                for line in range(len(groundtruth)):
                    id, gtt, st, et = [groundtruth['participant'].iloc[line], groundtruth['label'].iloc[line], int(groundtruth['starttime'].iloc[line]),
                                      int(groundtruth['endtime'].iloc[line])]
                    if starttime < st:
                        continue
                    else:
                        if (starttime > st) and (endtime < et):
                            result.append(gtt)
                        if result:
                            fea = list(dataset[fea_cols].iloc[gt])
                            inter_data = [p, st,et,find_majority(result)],(fea)
                            flatten = lambda l: [item for sublist in l for item in sublist]

                            data.append(flatten(inter_data))
    #         print(data)
df = pd.DataFrame(data)
df.fillna(df.mean(),inplace=True)
df.to_csv(_dirr + '/' + 'feature_ecg.csv', index=False, header=header)
df.shape

['.DS_Store' 'SI01' 'SI02' 'SI03' 'SI04' 'SI05' 'SI06' 'SI07' 'SI08'
 'SI09' 'SI10' 'SI11' 'SI12' 'SI13' 'SI14' 'SI15' 'SI16' 'SI17' 'SI18'
 'SI19' 'SI20' 'SI21' 'SI22' 'SI23' 'SI24' '.ipynb_checkpoints'
 'feature.csv' 'feature_rip.csv' 'feature_ecg.csv' 'feature_all.csv']


(2584, 15)

In [12]:
feature_file = './data/feature_ecg.csv'
feature = pd.read_csv(feature_file).values
y = np.int64(feature[:,3])
X = feature[:,4:]
print(X.shape,y.shape,np.sum(y))
groups = feature[:,0]

(2584, 11) (2584,) 492


In [13]:
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
m = len(np.where(y==0)[0])
n = len(np.where(y>0)[0])
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed

delta = 0.1

paramGrid = {'kernel': ['rbf'],
             'C': np.logspace(.1,4,5),
             'gamma': [np.power(2,np.float(x)) for x in np.arange(-4, 4, .5)],
             'class_weight': [{0: w, 1: 1 - w} for w in [.4,.3,.2]],
             'probability':[True]
}
# clf = Pipeline([('rf', SVC())])
clf = SVC()
gkf = GroupKFold(n_splits=len(np.unique(groups)))
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                           scoring='f1_weighted',verbose=5)
grid_search.fit(X,y)

print("Best parameter (CV score=%0.3f):" % grid_search.best_score_)
print(grid_search.best_params_)

import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups))
print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

Fitting 18 folds for each of 240 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 552 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 786 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 1056 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done 1362 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-1)]: Done 1704 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2082 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2496 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 2946 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 3432 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 3954 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:  4.2min finished


Best parameter (CV score=0.916):
{'C': 1.2589254117941673, 'class_weight': {0: 0.4, 1: 0.6}, 'gamma': 0.0625, 'kernel': 'rbf', 'probability': True}
[[2004   88]
 [ 130  362]]               precision    recall  f1-score   support

           0       0.94      0.96      0.95      2092
           1       0.80      0.74      0.77       492

    accuracy                           0.92      2584
   macro avg       0.87      0.85      0.86      2584
weighted avg       0.91      0.92      0.91      2584



In [14]:
import pickle
clf.set_params(probability=True)
print(clf)
clf.fit(X,y)
pickle.dump(clf,open('/home/jupyter/mullah/cc3/ecg_model_feature_standardization.p','wb'))

SVC(C=1.2589254117941673, break_ties=False, cache_size=200,
    class_weight={0: 0.4, 1: 0.6}, coef0=0.0, decision_function_shape='ovr',
    degree=3, gamma=0.0625, kernel='rbf', max_iter=-1, probability=True,
    random_state=None, shrinking=True, tol=0.001, verbose=False)


In [ ]:
from sklearn_porter import Porter
porter = Porter(clf, language='java')
output = porter.export(export_data=True)
# print(output)
text_file = open("SVM1.java", "w")
text_file.write(output)
text_file.close()
print(clf.probA_,clf.probB_)

In [10]:
from sklearn_porter import Porter

ModuleNotFoundError: No module named 'sklearn_porter'